In [1]:
from glob import glob
import os
import mne 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
all_file_path = glob(r"E:\A\python\EEG\dataverse_files\*.edf")
print(len(all_file_path))

28


In [3]:
all_file_path[0]

'E:\\A\\python\\EEG\\dataverse_files\\h01.edf'

In [4]:
def read_edf(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq = 0.5, h_freq = 45)
    epochs = mne.make_fixed_length_epochs(data, duration=5, overlap= 1)
    array = epochs.get_data()
    return array

In [5]:
healthy_file_path = [i for i in all_file_path if 'h' in os.path.basename(i)]
patient_file_path = [i for i in all_file_path if 's' in os.path.basename(i)]

print(len(healthy_file_path), len(patient_file_path))

14 14


In [19]:
def read_edf(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq = 0.5, h_freq = 45)
    epochs = mne.make_fixed_length_epochs(data, duration=5, overlap= 1)
    array = epochs.get_data()
    return array

In [6]:
sample_data = read_edf(healthy_file_path[0])

Extracting EDF parameters from E:\A\python\EEG\dataverse_files\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
231 matching events found
No baseline correction applied
0 projection items activated
Usin

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [7]:
sample_data.shape

(231, 19, 1250)

In [8]:
%%capture

control_epochs_array = [read_edf(i) for i in healthy_file_path]
patient_epochs_array = [read_edf(i) for i in patient_file_path]

In [9]:
control_epochs_array[0].shape

(231, 19, 1250)

In [10]:
control_epochs_labels = [[len(i)] for i in control_epochs_array]
patient_epochs_labels = [[len(i)] for i in patient_epochs_array]

len(control_epochs_labels), len(patient_epochs_labels)

(14, 14)

In [11]:
data_list = control_epochs_array + patient_epochs_array

label_list = control_epochs_labels + patient_epochs_labels

In [12]:
group_list = [[i] for i, j in enumerate(data_list)]
len(group_list)

28

In [ ]:
data_array = np.vstack(data_list)

label_array = np.hstack(label_list)

group_array = np.hstack(group_list)

print(data_array.shape, label_array.shape, group_array.shape)

(7201, 19, 1250) (28,) (28,)


feature extraction 

In [16]:
np.mean(data_array, axis=0).shape

(19, 1250)

In [19]:
from scipy import stats

def mean(x):
    return np.mean(x, axis=-1)

def std(x):
    return np.std(x, axis=-1)

def ptp(x):
    return np.ptp(x, axis=-1)

def var(x):
    return np.var(x, axis=-1)

def minim(x):
    return np.min(x, axis=-1)

def maxim(x):
    return np.max(x, axis=-1)

def argmin(x):
    return np.argmin(x, axis=-1)

def argmax(x):
    return np.argmax(x, axis=-1)

def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1))

def abs_dif_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1)

def skewness(x):
    return stats.skew(x, axis=-1)

def kurtosis(x):
    return stats.kurtosis(x, axis=-1)

def concatenate_features(x):
    return np.concatenate([mean(x), std(x), ptp(x), var(x), minim(x), maxim(x), argmin(x), argmax(x), rms(x), abs_dif_signal(x), skewness(x), kurtosis(x)], axis=-1)

In [20]:
features = [] 

for d in data_array:
    features.append(concatenate_features(d))

In [21]:
features_array = np.vstack(features)
features_array.shape

(7201, 228)

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV

In [37]:
clf = LogisticRegression()
# gk
gk = GroupKFold(5)
# pipe
pipe = Pipeline([('scaler', StandardScaler()), ('clf', clf)])
# param_grid
param_grid = {'clf__C': [0.01, 0.05, 0.1, 0.5, 1, 3, 5, 7, 10]}

# Create expanded group array that matches the length of features_array
group_array_expanded = []
for i, label in enumerate(label_list):
	group_array_expanded.extend([i] * label[0])  # Use label[0] since each label is a single-element list
group_array_expanded = np.array(group_array_expanded)

# gscv
gscv = GridSearchCV(pipe, param_grid, cv=gk, n_jobs=6)
gscv.fit(features_array, label_array, groups=group_array_expanded)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf', LogisticRegression())]),
             n_jobs=6,
             param_grid={'clf__C': [0.01, 0.05, 0.1, 0.5, 1, 3, 5, 7, 10]})

In [38]:
gscv.best_score_

0.05539494363606499

In [1]:
import matplotlib 
import pathlib 
import mne 

In [2]:

# Specify the Agg backend before importing matplotlib.pyplot.
import matplotlib
matplotlib.use("Agg")  # Set the backend to Agg (non-interactive, file-based)

# Import other necessary libraries.
import pathlib
import mne
import matplotlib.pyplot as plt # Need to import pyplot to get the backend

print(f"Current backend is: {matplotlib.get_backend()}")

Current backend is: Agg


In [3]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = pathlib.Path(sample_data_folder) / "MEG" / "sample" / "sample_audvis_filt-0-40_raw.fif"

Using default location ~/mne_data for sample...
Creating C:\Users\USER\mne_data


100%|#############################################| 1.65G/1.65G [00:00<?, ?B/s]
Untarring contents of 'C:\Users\USER\mne_data\MNE-sample-data-processed.tar.gz' to 'C:\Users\USER\mne_data'


Attempting to create new mne-python configuration file:
C:\Users\USER\.mne\mne-python.json
Download complete in 06m42s (1576.2 MB)


In [4]:
raw = mne.io.read_raw(sample_data_raw_file)
raw 

Opening raw data file C:\Users\USER\mne_data\MNE-sample-data\MEG\sample\sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.


<Raw | sample_audvis_filt-0-40_raw.fif, 376 x 41700 (277.7 s), ~3.2 MiB, data not loaded>

In [8]:
# how can i show the plotting that i download now  

# Plot the raw data.
raw.plot()



c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\mne\viz\utils.py:158: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  (fig or plt).show(**kwargs)


<MNEBrowseFigure size 800x800 with 5 Axes>